<a href="https://colab.research.google.com/github/zw2497/Twitter_Stream_Processing/blob/master/TweepyPush2Kafka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tweepy
!pip install kafka-python

In [1]:
import tweepy

In [2]:
from kafka import KafkaProducer, KafkaConsumer
from kafka.errors import KafkaError
from json import dumps, loads

In [3]:
producer = KafkaProducer(bootstrap_servers=['35.243.144.79:9092'],
                         value_serializer=lambda x: dumps(x).encode('utf-8'))

### Setup Streaming

In [4]:
class LDSStreamListener(tweepy.StreamListener):
    def on_status(self, status):
#         print(status.text)
        producer.send('tweepyv1', value=status._json) 

In [5]:
consumer_key  = "2wrYTQnh4poBbF5MsubR3sk4kzw"
consumer_secret = "93Yi0isNNgNPW4bFo8hsMpANUIw61lofgRRpz8H39P5V6o63tlzw"
access_token  = "2761651876-LoLkgE3EFYkqIHSGEPvzB3udtmATj4XNem2MemWzw"
access_token_secret = "1uDlUpdk6mrinFKFl3vN06xIVeORaa60EjubXrEOSpYKczw"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

myStream = tweepy.Stream(auth=api.auth, listener=LDSStreamListener())

### Start Streaming

In [ ]:
myStream.filter(locations=(-171, 18, -66, 71), languages=['en'])

### Consumer Test

In [ ]:
consumer = KafkaConsumer(
    'fast',
     bootstrap_servers=['35.243.144.79:9092'],
     auto_offset_reset='earliest',
     enable_auto_commit=False,
     group_id='my-group3102',
     value_deserializer=lambda x: loads(x.decode('utf-8')))

In [ ]:
for message in consumer:
    message = message.value
    print(message)